In [1]:
# Inisialisasi Mediapipe
import cv2
import mediapipe as mp
import numpy as np
import time

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Fungsi untuk menghitung kesamaan horizontal
def calculate_similarity(img_left, img_right):
    height, width = img_left.shape[:2]
    left_gray = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    # Normalisasi ukuran
    right_resized = cv2.resize(right_gray, (width, height))

    # Hitung perbedaan absolut
    diff = cv2.absdiff(left_gray, right_resized)
    score = 100 - np.mean(diff) * 100 / 255
    return max(0, min(100, score))

# Daftar gambar yang sudah ada

# Fungsi untuk menggambar kotak teks
def draw_instruction_box(frame, text, countdown):
    h, w, _ = frame.shape
    box_width, box_height = 610, 150
    top_left_x = (w - box_width) // 2
    top_left_y = max(0, int(h * 0.2))
    bottom_right_x = top_left_x + box_width
    bottom_right_y = top_left_y + box_height

    # Kotak teks hijau dengan sisi rounded dan border hijau gelap
    overlay = frame.copy()
    cv2.rectangle(overlay, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 128, 0), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)

    # Membagi teks menjadi dua baris jika terlalu panjang
    if len(text) > 50:
        split_index = text[:50].rfind(" ")
        text1 = text[:split_index]
        text2 = text[split_index+1:]
        cv2.putText(frame, text1, (top_left_x + 20, top_left_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, text2, (top_left_x + 20, top_left_y + 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    else:
        cv2.putText(frame, text, (top_left_x + 20, top_left_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.putText(frame, str(countdown), (top_left_x + box_width // 2 - 20, top_left_y + 120), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

# Buka kamera
cap = cv2.VideoCapture(0)
session = 1
countdown = 5
countdown_start_time = None
stop_time = None
is_nodding = False
accuracy_displayed = False
nilaiAkurasi = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if session == 1:  # Sesi 1: Kotak Teks
        if countdown_start_time is None:
            countdown_start_time = time.time()

        elapsed_time = time.time() - countdown_start_time
        if elapsed_time >= 1:
            countdown -= 1
            countdown_start_time = time.time()

        if results.multi_face_landmarks:
            draw_instruction_box(frame, "Anggukan Kepalamu Untuk Mengetes Seberapa Cepat Kamu", countdown)

        if countdown <= 0:
            session = 2
            countdown_start_time = None

    elif session == 2:  # Sesi 2: Gambar Berputar
        img_path = np.random.choice(existing_images)
        image = cv2.imread(img_path)

        # Membelah gambar
        mid_x = image.shape[1] // 2
        img_left = image[:, :mid_x]
        img_right = image[:, mid_x:]

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                forehead_x = int(face_landmarks.landmark[10].x * width)
                forehead_y = int(face_landmarks.landmark[10].y * height) - 50  # Posisi di atas dahi

                # Validasi dimensi koordinat untuk img_left
                if 0 <= forehead_y < height - img_left.shape[0] and 0 <= forehead_x - img_left.shape[1] < width:
                    frame[forehead_y:forehead_y+img_left.shape[0], forehead_x-img_left.shape[1]:forehead_x] = img_left

                # Menempatkan sisi kanan gambar
                spacing = int(0.03 * width)  # Jarak antar gambar (3 cm dalam piksel)
                if 0 <= forehead_y < height - img_right.shape[0] and 0 <= forehead_x + spacing < width - img_right.shape[1]:
                    frame[forehead_y:forehead_y+img_right.shape[0], forehead_x+spacing:forehead_x+spacing+img_right.shape[1]] = img_right

                # Deteksi anggukan
                top_of_forehead = face_landmarks.landmark[10]
                chin = face_landmarks.landmark[152]
                top_y = int(top_of_forehead.y * height)
                chin_y = int(chin.y * height)

                if chin_y - top_y > 50:  # Acuan anggukan 5cm
                    is_nodding = True
                    if stop_time is None:
                        stop_time = time.time()

                if is_nodding and stop_time and time.time() - stop_time > 1:
                    nilaiAkurasi = calculate_similarity(img_left, img_right)
                    session = 3
                    time.sleep(5)  # Jeda sebelum ke sesi 3

    elif session == 3:  # Sesi 3: Hasil
        if nilaiAkurasi is not None:
            accuracy_text = f"Nilai keakuratan kamu sebesar {nilaiAkurasi:.2f}%"
            text_size = cv2.getTextSize(accuracy_text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
            text_x = (width - text_size[0]) // 2
            text_y = height // 2
            cv2.putText(frame, accuracy_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            accuracy_displayed = True

    cv2.imshow("Reaction Time", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Tekan ESC untuk keluar
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'os' is not defined